In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('elu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 28s 81ms/step - loss: 1.9193 - accuracy: 0.3111 - val_loss: 1.5840 - val_accuracy: 0.4628
Epoch 2/50
313/313 [==============================] - 24s 75ms/step - loss: 1.5089 - accuracy: 0.4786 - val_loss: 1.7317 - val_accuracy: 0.4016
Epoch 3/50
313/313 [==============================] - 22s 72ms/step - loss: 1.3796 - accuracy: 0.5249 - val_loss: 1.3326 - val_accuracy: 0.5197
Epoch 4/50
313/313 [==============================] - 25s 81ms/step - loss: 1.2784 - accuracy: 0.5660 - val_loss: 1.3751 - val_accuracy: 0.5178
Epoch 5/50
313/313 [==============================] - 23s 72ms/step - loss: 1.2161 - accuracy: 0.5876 - val_loss: 1.2359 - val_accuracy: 0.5689
Epoch 6/50
313/313 [==============================] - 23s 72ms/step - loss: 1.1418 - accuracy: 0.6117 - val_loss: 1.2196 - val_accuracy: 0.5709
Epoch 7/50
313/313 [==============================] - 25s 79ms/step - loss: 1.0872 - accuracy: 0.6350 - val_loss: 1.2841 - val_accuracy:

In [10]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.7372740507125854, 1.477248191833496, 1.356330394744873, 1.2668501138687134, 1.198671817779541, 1.1362309455871582, 1.0789896249771118, 1.0314744710922241, 0.9850724339485168, 0.9501058459281921, 0.9077738523483276, 0.8708226680755615, 0.8398371934890747, 0.803415060043335, 0.7642993927001953, 0.7357632517814636, 0.7053939700126648, 0.6719874739646912, 0.6385971903800964, 0.6112286448478699, 0.5779175162315369, 0.5512904524803162, 0.5273371338844299, 0.4926273822784424, 0.47020453214645386, 0.4433380365371704, 0.41997450590133667, 0.3992844820022583, 0.37573760747909546, 0.35727718472480774, 0.32707324624061584, 0.30899739265441895, 0.2904185652732849, 0.27229300141334534, 0.24852456152439117, 0.23058339953422546, 0.21668986976146698, 0.20325329899787903, 0.18951033055782318, 0.17503438889980316, 0.1627914011478424, 0.15041927993297577, 0.1404515504837036, 0.13157695531845093, 0.1263136863708496, 0.11649037897586823, 0.10700376331806183, 0.10150459408760071, 0.09410199522972